In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
pd.get_option("display.max_columns")
pd.set_option('display.max_columns', 200)

DATA_DIR = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021'

import os, sys
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/submission_xgb.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/MNCAATourneyDetailedResults.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/submission_svc.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/tourney_rato.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/MNCAATourneyCompactResults.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/test_preds.npy
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/MSeasons.csv
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/eda copy.ipynb
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/.DS_Store
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/basketball_xgboost.ipynb
/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/ncaam-march-mania-2021/b

In [4]:
MNCAATourneyCompactResults = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')
MNCAATourneyCompactResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [69]:
MNCAATourneySeeds = pd.read_csv(DATA_DIR+'/MNCAATourneySeeds.csv')
MNCAATourneySeeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [68]:
MNCAATourneySlots = pd.read_csv(DATA_DIR+'/MNCAATourneySlots.csv')
MNCAATourneySlots

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12
...,...,...,...,...
2246,2019,R6CH,R5WX,R5YZ
2247,2019,W11,W11a,W11b
2248,2019,W16,W16a,W16b
2249,2019,X11,X11a,X11b


In [46]:
MRSCResults = pd.read_csv(DATA_DIR + '/MRegularSeasonCompactResults.csv')
MRSCResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
166875,2020,128,1204,81,1209,62,A,0
166876,2020,128,1402,85,1111,68,H,0
166877,2020,128,1299,86,1224,77,N,0
166878,2020,128,1393,81,1314,53,N,0


In [47]:
A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)
win

win_A  win_N  win_H
Season WTeamID                     
1985   1102       3.0    0.0    2.0
       1103       2.0    0.0    7.0
       1104       5.0    1.0   15.0
       1106       1.0    1.0    8.0
       1108       8.0    3.0    8.0
...               ...    ...    ...
2020   1463      11.0    2.0    8.0
       1464       5.0    0.0   11.0
       1465       6.0    0.0   11.0
       1466       4.0    0.0    7.0
       1467       9.0    1.0    9.0

[11580 rows x 3 columns]

In [48]:
win.index = win.index.rename(['Season', 'TeamID'])
win

win_A  win_N  win_H
Season TeamID                     
1985   1102      3.0    0.0    2.0
       1103      2.0    0.0    7.0
       1104      5.0    1.0   15.0
       1106      1.0    1.0    8.0
       1108      8.0    3.0    8.0
...              ...    ...    ...
2020   1463     11.0    2.0    8.0
       1464      5.0    0.0   11.0
       1465      6.0    0.0   11.0
       1466      4.0    0.0    7.0
       1467      9.0    1.0    9.0

[11580 rows x 3 columns]

In [49]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
A_scr

<ipython-input-49-616075a97464>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\


Score_A  relScore_A
Season TeamID                       
1985   1102    62.833333   -8.166667
       1103    60.800000   -8.300000
       1104    65.181818    4.000000
       1106    72.272727   -7.272727
       1108    78.583333    1.583333
...                  ...         ...
2020   1463    72.058824    4.647059
       1464    69.125000   -6.875000
       1465    68.090909   -0.181818
       1466    69.294118   -7.235294
       1467    59.000000   -4.000000

[11594 rows x 2 columns]

FGM - field goals made (by the winning team)<br>
FGA - field goals attempted (by the winning team)<br>
FGM3 - three pointers made (by the winning team)<br>
FGA3 - three pointers attempted (by the winning team)<br>
FTM - free throws made (by the winning team)<br>
FTA - free throws attempted (by the winning team)<br>
OR - offensive rebounds (pulled by the winning team)<br>
DR - defensive rebounds (pulled by the winning team)<br>
Ast - assists (by the winning team)<br>
TO - turnovers committed (by the winning team)<br>
Stl - steals (accomplished by the winning team)<br>
Blk - blocks (accomplished by the winning team)<br>
PF - personal fouls committed (by the winning team)<br>

In [50]:
MRSDetailedResults = pd.read_csv(DATA_DIR + '/MRegularSeasonDetailedResults.csv')
MRSDetailedResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92827,2020,128,1389,56,1233,54,N,0,15,60,5,24,21,26,19,26,4,14,6,5,23,17,46,2,16,18,20,4,27,6,17,8,4,20
92828,2020,128,1393,81,1314,53,N,0,28,61,7,21,18,28,10,29,15,8,11,3,17,20,60,2,16,11,13,13,29,9,18,0,1,18
92829,2020,128,1402,85,1111,68,H,0,34,57,4,12,13,16,6,25,20,14,9,0,24,20,46,3,15,25,35,8,17,6,18,7,2,18
92830,2020,128,1412,74,1427,69,N,0,26,53,8,14,14,20,8,29,12,16,6,5,16,26,67,10,27,7,7,12,22,11,13,4,2,18


In [52]:
MMOrdinals = pd.read_csv(DATA_DIR + '/MMasseyOrdinals.csv')
MMOrdinals

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260
...,...,...,...,...,...
4120881,2020,128,WOL,1463,53
4120882,2020,128,WOL,1464,209
4120883,2020,128,WOL,1465,194
4120884,2020,128,WOL,1466,290


In [53]:
TCResults = pd.read_csv(DATA_DIR + '/MNCAATourneyCompactResults.csv')
TCResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [54]:
MSampleSubmission = pd.read_csv(DATA_DIR + '/MSampleSubmissionStage1.csv')
MSampleSubmission

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5
...,...,...
11385,2019_1449_1459,0.5
11386,2019_1449_1463,0.5
11387,2019_1458_1459,0.5
11388,2019_1458_1463,0.5


In [55]:
test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1

,ID,Pred,Season
0,2015_1107_1112,0.5,2015
1,2015_1107_1116,0.5,2015
2,2015_1107_1124,0.5,2015
3,2015_1107_1125,0.5,2015
4,2015_1107_1129,0.5,2015
...,...,...,...
11385,2019_1449_1459,0.5,2019
11386,2019_1449_1463,0.5,2019
11387,2019_1458_1459,0.5,2019
11388,2019_1458_1463,0.5,2019


In [56]:
train = pd.read_csv(DATA_DIR+'/tourney.csv')
train

,Season,TeamID1,TeamID2,result,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,win_pct_H_2,win_pct_All_2,Score_A_1,relScore_A_1,Score_N_1,relScore_N_1,Score_H_1,relScore_H_1,Score_All_1,relScore_All_1,Score_A_2,relScore_A_2,Score_N_2,relScore_N_2,Score_H_2,relScore_H_2,Score_All_2,relScore_All_2,FGM_1,FGA_1,FGM3_1,FGA3_1,FTM_1,FTA_1,OR_1,DR_1,Ast_1,TO_1,Stl_1,Blk_1,PF_1,goal_rate_1,3p_goal_rate_1,ft_goal_rate_1,total_attack_1,Points_Per_Possession_1,POSS_1,ORTG_1,DRTG_1,NRTG_1,eFG%_1,FGM2_1,PPS_1,FGA2_1,2PPS_1,3PPS_1,TOV%_1,OREB%_1,DREB%_1,FTR_1,TS%_1,3P/2P_1,AST%_1,AST/TO_1,FGM_2,FGA_2,FGM3_2,FGA3_2,FTM_2,FTA_2,OR_2,DR_2,Ast_2,TO_2,Stl_2,Blk_2,PF_2,goal_rate_2,3p_goal_rate_2,ft_goal_rate_2,total_attack_2,Points_Per_Possession_2,POSS_2,ORTG_2,DRTG_2,NRTG_2,eFG%_2,FGM2_2,PPS_2,FGA2_2,2PPS_2,3PPS_2,TOV%_2,OREB%_2,DREB%_2,FTR_2,TS%_2,3P/2P_2,AST%_2,AST/TO_2,OrdinalRank_127_128_1,maxRank_127_128_1,RankMax_1,RankMin_1,RankStd_1,RankMean_1,RankTrans_50_51_to_127_128_1,RankTrans_15_16_to_127_128_1,OrdinalRank_127_128_2,maxRank_127_128_2,RankMax_2,RankMin_2,RankStd_2,RankMean_2,RankTrans_50_51_to_127_128_2,RankTrans_15_16_to_127_128_2,OrdinalRank_127_128_diff
0,2003,1421,1411,1,0.125000,1.000000,0.833333,0.448276,0.333333,0.750000,0.785714,0.600000,64.750000,-15.750000,81.000000,1.000000,79.000000,3.416667,71.206897,-7.241379,67.000000,-4.583333,83.000000,4.500000,74.857143,6.857143,72.800000,1.966667,24.379310,56.793103,6.482759,18.000000,15.965517,20.931034,12.275862,23.172414,13.034483,16.206897,7.068966,3.000000,19.103448,0.432768,0.363494,0.766142,82.000345,0.873327,69.933793,102.202638,111.999033,-9.796395,0.489800,1.931034,0.404170,17.862069,35.724138,1.090482,0.230355,0.347184,0.625619,0.377214,0.541825,1.176868,0.524912,0.916938,24.733333,55.266667,5.933333,18.500000,17.400000,28.066667,13.166667,24.800000,14.200000,15.233333,6.433333,2.233333,18.300000,0.448892,0.321414,0.613745,82.568667,0.881633,69.682667,104.375757,102.302227,2.073530,0.503036,1.400000,0.345695,8.700000,17.400000,0.964242,0.218210,0.360133,0.681723,0.529491,0.539819,2.429609,0.574708,0.983780,0.850153,327.0,296.0,214.0,25.951032,267.928571,0.088685,-1.000000,0.917431,327.0,324.0,284.0,12.782199,307.000000,-0.073394,-1.000000,-0.067278
1,2003,1112,1436,1,0.916667,0.000000,0.933333,0.892857,0.500000,0.600000,0.900000,0.655172,82.416667,12.000000,89.000000,-7.000000,87.200000,18.800000,85.214286,14.964286,63.142857,-1.714286,71.200000,6.800000,72.600000,12.500000,67.793103,4.655172,30.321429,65.714286,7.035714,20.071429,17.535714,25.000000,15.178571,27.642857,17.642857,14.785714,8.464286,4.214286,17.750000,0.463563,0.351060,0.701154,91.250000,0.936347,76.321429,111.912288,92.372397,19.539891,0.517632,5.750000,0.490835,20.642857,41.285714,1.053181,0.192304,0.394027,0.680560,0.387965,0.557334,0.463458,0.579706,1.293499,24.827586,55.862069,5.275862,15.482759,12.862069,19.551724,12.965517,25.724138,14.206897,14.068966,6.862069,2.965517,15.896552,0.446372,0.357920,0.649708,78.338276,0.866624,65.568276,103.714356,96.266018,7.448338,0.494732,6.689655,0.519295,20.827586,41.655172,1.073759,0.215063,0.380016,0.732898,0.356274,0.528383,1.228217,0.568251,1.077472,0.009174,327.0,4.0,1.0,0.825420,2.714286,0.000000,-1.000000,0.574924,327.0,208.0,176.0,9.740873,195.500000,-0.061162,-1.000000,-0.565749
2,2003,1113,1272,1,0.400000,0.500000,0.800000,0.620690,0.636364,1.000000,0.875000,0.793103,72.200000,-0.700000,75.000000,0.750000,78.733333,13.400000,75.965517,6.793103,74.000000,2.090909,66.000000,6.500000,75.937500,13.500000,74.517241,8.689655,27.206897,56.896552,4.000000,12.586207,17.551724,26.206897,13.689655,23.310345,15.551724,14.000000,5.206897,4.241379,19.413793,0.481680,0.328376,0.675667,82.165517,0.927195,68.737931,110.721516,100.617574,10.103941,0.517334,5.655172,0.408128,18.103448,36.206897,0.985129,0.203347,0.398080,0.683950,0.468561,0.557354,1.359494,0.572377,1.203251,26.275862,60.000000,7.000000,20.068966,14.965517,22.896552,14.068966,2

In [57]:
train.result.value_counts()

1    780
0    780
Name: result, dtype: int64

In [60]:
corr = train.corr()

In [67]:
corr.sort_values(by='Score_A_1', ascending=False).tail(20)

,Season,TeamID1,TeamID2,result,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,win_pct_H_2,win_pct_All_2,Score_A_1,relScore_A_1,Score_N_1,relScore_N_1,Score_H_1,relScore_H_1,Score_All_1,relScore_All_1,Score_A_2,relScore_A_2,Score_N_2,relScore_N_2,Score_H_2,relScore_H_2,Score_All_2,relScore_All_2,FGM_1,FGA_1,FGM3_1,FGA3_1,FTM_1,FTA_1,OR_1,DR_1,Ast_1,TO_1,Stl_1,Blk_1,PF_1,goal_rate_1,3p_goal_rate_1,ft_goal_rate_1,total_attack_1,Points_Per_Possession_1,POSS_1,ORTG_1,DRTG_1,NRTG_1,eFG%_1,FGM2_1,PPS_1,FGA2_1,2PPS_1,3PPS_1,TOV%_1,OREB%_1,DREB%_1,FTR_1,TS%_1,3P/2P_1,AST%_1,AST/TO_1,FGM_2,FGA_2,FGM3_2,FGA3_2,FTM_2,FTA_2,OR_2,DR_2,Ast_2,TO_2,Stl_2,Blk_2,PF_2,goal_rate_2,3p_goal_rate_2,ft_goal_rate_2,total_attack_2,Points_Per_Possession_2,POSS_2,ORTG_2,DRTG_2,NRTG_2,eFG%_2,FGM2_2,PPS_2,FGA2_2,2PPS_2,3PPS_2,TOV%_2,OREB%_2,DREB%_2,FTR_2,TS%_2,3P/2P_2,AST%_2,AST/TO_2,OrdinalRank_127_128_1,maxRank_127_128_1,RankMax_1,RankMin_1,RankStd_1,RankMean_1,RankTrans_50_51_to_127_128_1,RankTrans_15_16_to_127_128_1,OrdinalRank_127_128_2,maxRank_127_128_2,RankMax_2,RankMin_2,RankStd_2,RankMean_2,RankTrans_50_51_to_127_128_2,RankTrans_15_16_to_127_128_2,OrdinalRank_127_128_diff
Score_All_2,-1.318691e-01,0.009728,-0.069983,-0.169143,0.018035,-0.014639,0.014408,0.007488,0.291219,0.085393,0.248589,0.351358,-0.037969,-0.008980,-0.032296,-0.020948,-0.033326,-0.029126,-0.048234,-0.031945,0.866971,0.392069,0.404821,0.168724,0.931807,0.554468,1.000000,0.567237,-0.045879,-0.042647,-0.029381,-0.021537,-0.011408,-0.002509,-0.016479,-0.039025,-0.015974,0.027762,-0.013737,-0.037120,0.042601,-0.017360,-0.035516,-0.035936,-0.023339,-0.040297,-0.022602,-0.048051,-0.015459,-0.025442,-0.021233,-0.012489,-0.018066,-0.019271,-0.019271,-0.035516,0.043984,-0.011588,0.002976,0.017969,-0.021946,-0.041514,0.019092,-0.025445,0.906302,0.738672,0.283171,0.164888,0.529517,0.494644,0.426618,0.452529,0.603451,0.148066,0.341961,0.267118,0.137972,0.499270,0.366430,0.158271,0.750467,0.539472,0.783117,0.660278,0.027765,0.477870,0.454916,0.154031,0.117546,0.183103,0.183103,0.366430,-0.188862,0.340552,-0.143810,0.116502,0.476938,-0.015412,0.048250,0.267296,0.049402,-9.751027e-02,0.055380,0.052637,0.046013,0.052841,-0.048590,-0.072355,-0.373141,-9.751027e-02,-0.376185,-0.353801,-0.298354,-0.371190,-0.016457,-0.063052,3.015051e-01
Score_N_2,-3.917779e-02,-0.006104,-0.038111,-0.125347,-0.036255,0.006565,-0.018895,-0.037553,0.103417,0.701020,0.098689,0.163350,-0.038095,-0.037447,0.018847,-0.002001,-0.025963,-0.045882,-0.032296,-0.043528,0.290564,0.147458,1.000000,0.354566,0.340849,0.254448,0.404821,0.271286,-0.025957,-0.024658,-0.017746,0.002476,-0.018334,-0.001920,-0.037251,-0.034618,-0.026361,0.022210,-0.006701,-0.014343,0.056270,-0.002761,-0.042058,-0.059134,-0.011368,-0.030158,0.004001,-0.052979,0.003054,-0.042011,-0.005874,0.002771,-0.000507,-0.021876,-0.021876,-0.042058,0.025146,-0.047323,-0.043929,0.012419,-0.013988,-0.016234,-0.012828,-0.031166,0.388208,0.329637,0.114033,0.082266,0.172237,0.163849,0.210853,0.117736,0.248916,-0.021340,0.167499,0.123132,0.053446,0.192774,0.117620,0.041986,0.288284,0.233464,0.276484,0.311626,-0.005424,0.238406,0.170332,0.112194,0.091410,0.113709,0.113709,0.117620,-0.147602,0.188933,-0.106934,-0.009127,0.165657,0.010058,0.007762,0.154810,0.040536,-2.963238e-02,0.033919,0.034806,0.025832,0.031915,0.020442,-0.062060,-0.277120,-2.963238e-02,-0.294544,-0.272397,-0.237796,-0.293219,0.041001,-0.048820,2.266629e-01
total_attack_2,-2.390147e-01,0.006105,-0.047745,-0.074499,0.009348,-0.021120,0.009957,-0.005935,0.081341,-0.016481,0.083755,0.093234,-0.038497,-0.020204,-0.011368,-0.029329,0.002838,-0.007953,-0.023339,-0.024034,0.668151,0.093139,0.288284,-0.029580,0.699156,0.252383,0.750467,0.204591,-0.017724,-0.017400,-0.030252,-0.028809,-0.004427,-0.006039,-0.006202,-0.016237,0.019216,0.076335,-0.004928,-0.035921,0.036503,-0.003726,-0.011636,-0.005545,0.012920,-0.046812,0.020325,-0.058544,-0.030449,-0.022931,-0.012566,0.004420,-0.011927,0.0

In [2]:
#get elo data from warrennolan.com - Great resource (check it out!)
df_elo = pd.read_html('http://warrennolan.com/basketball/2021/elochess')[0]

df_elo_final = df_elo.loc[:,['Team','ELO']]
df_elo_final.head()

,Team,ELO
0,Gonzaga,1870.14
1,Illinois,1811.07
2,Baylor,1809.62
3,Alabama,1753.74
4,Houston,1748.56


In [3]:
 df_elo

,Unnamed: 0,Team,Record,ELO,Rank,ELO Delta
0,NaN,Gonzaga,26 - 0,1870.14,1,NaN
1,NaN,Illinois,24 - 6,1811.07,2,NaN
2,NaN,Baylor,23 - 2,1809.62,3,NaN
3,NaN,Alabama,24 - 6,1753.74,4,NaN
4,NaN,Houston,24 - 3,1748.56,5,NaN
...,...,...,...,...,...,...
352,NaN,South Carolina State,1 - 17,961.53,353,NaN
353,NaN,Delaware State,1 - 16,954.15,354,NaN
354,NaN,Idaho,1 - 21,952.32,355,NaN
355,NaN,Mississippi Valley State,2 - 22,938.09,356,NaN


In [10]:
import pandas as pd
df = pd.read_html('https://kenpom.com')[0]
df

Unnamed: 0_level_0      Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk                    Team                Conf   
     Unnamed: 0_level_2      Unnamed: 1_level_2  Unnamed: 2_level_2   
                     Rk                    Team                Conf   
     Unnamed: 0_level_4      Unnamed: 1_level_4  Unnamed: 2_level_4   
                     Rk                    Team                Conf   
     Unnamed: 0_level_6      Unnamed: 1_level_6  Unnamed: 2_level_6   
                     Rk                    Team                Conf   
     Unnamed: 0_level_8      Unnamed: 1_level_8  Unnamed: 2_level_8   
                     Rk                    Team                Conf   
    Unnamed: 0_level_10     Unnamed: 1_level_10 Unnamed: 2_level_10   
                     Rk                    Team                Conf   
    Unnamed: 0_level_12     Unnamed: 1_level_12 Unnamed: 2_level_12   
                     Rk                    Team                Conf   
    Unnamed: 0_level_14     Unnamed: 1_level_14 Unnamed: 2_level_14   
                     Rk                    Team                Conf   
    Unnamed: 0_level_16     Unnamed: 1_level_16 Unnamed: 2_level_16   
                     Rk                    Team                Conf   
0                     1               Gonzaga 1                 WCC   
1                     2              Illinois 1                 B10   
2                     3              Michigan 1                 B10   
3                     4                Baylor 1                 B12   
4                     5               Houston 2                Amer   
..                  ...                     ...                 ...   
368                 353      South Carolina St.                MEAC   
369                 354             Alabama St.                SWAC   
370                 355  Maryland Eastern Shore                MEAC   
371                 356             Chicago St.                 WAC   
372                 357  Mississippi Valley St.                SWAC   

     Unnamed: 3_level_0  Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_2  Unnamed: 4_level_2  Unnamed: 5_level_2   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_4  Unnamed: 4_level_4  Unnamed: 5_level_4   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_6  Unnamed: 4_level_6  Unnamed: 5_level_6   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_8  Unnamed: 4_level_8  Unnamed: 5_level_8   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_10 Unnamed: 4_level_10 Unnamed: 5_level_10   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_12 Unnamed: 4_level_12 Unnamed: 5_level_12   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_14 Unnamed: 4_level_14 Unnamed: 5_level_14   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_16 Unnamed: 4_level_16 Unnamed: 5_level_16   
                    W-L               AdjEM                AdjO   
0                  26-0              +36.59               126.0   
1                  24-6              +30.70               118.8   
2                  20-4              +30.50               119.3   
3                  23-2              +29.93               123.0   
4                  25-3              +29.68               119.6   
..                  ...                 ...                 ...   
368                1-17              -25.02                87.1   
369                4-14              -26.12                84.2   
370                 0-0              -27.33                83.9   
371                 0-9              -29.52                88.3   
372                2-22              -41.51                79.2   

 

In [22]:
df.columns[0]

('Unnamed: 0_level_0',
 'Rk',
 'Unnamed: 0_level_2',
 'Rk',
 'Unnamed: 0_level_4',
 'Rk',
 'Unnamed: 0_level_6',
 'Rk',
 'Unnamed: 0_level_8',
 'Rk',
 'Unnamed: 0_level_10',
 'Rk',
 'Unnamed: 0_level_12',
 'Rk',
 'Unnamed: 0_level_14',
 'Rk',
 'Unnamed: 0_level_16',
 'Rk')

In [27]:
df.rename(columns={'Unnamed: 0_level_0':'Rk'}, inplace=True)
df

Rk      Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk                    Team                Conf   
     Unnamed: 0_level_2      Unnamed: 1_level_2  Unnamed: 2_level_2   
                     Rk                    Team                Conf   
     Unnamed: 0_level_4      Unnamed: 1_level_4  Unnamed: 2_level_4   
                     Rk                    Team                Conf   
     Unnamed: 0_level_6      Unnamed: 1_level_6  Unnamed: 2_level_6   
                     Rk                    Team                Conf   
     Unnamed: 0_level_8      Unnamed: 1_level_8  Unnamed: 2_level_8   
                     Rk                    Team                Conf   
    Unnamed: 0_level_10     Unnamed: 1_level_10 Unnamed: 2_level_10   
                     Rk                    Team                Conf   
    Unnamed: 0_level_12     Unnamed: 1_level_12 Unnamed: 2_level_12   
                     Rk                    Team                Conf   
    Unnamed: 0_level_14     Unnamed: 1_level_14 Unnamed: 2_level_14   
                     Rk                    Team                Conf   
    Unnamed: 0_level_16     Unnamed: 1_level_16 Unnamed: 2_level_16   
                     Rk                    Team                Conf   
0                     1               Gonzaga 1                 WCC   
1                     2              Illinois 1                 B10   
2                     3              Michigan 1                 B10   
3                     4                Baylor 1                 B12   
4                     5               Houston 2                Amer   
..                  ...                     ...                 ...   
368                 353      South Carolina St.                MEAC   
369                 354             Alabama St.                SWAC   
370                 355  Maryland Eastern Shore                MEAC   
371                 356             Chicago St.                 WAC   
372                 357  Mississippi Valley St.                SWAC   

     Unnamed: 3_level_0  Unnamed: 4_level_0  Unnamed: 5_level_0  \
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_2  Unnamed: 4_level_2  Unnamed: 5_level_2   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_4  Unnamed: 4_level_4  Unnamed: 5_level_4   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_6  Unnamed: 4_level_6  Unnamed: 5_level_6   
                    W-L               AdjEM                AdjO   
     Unnamed: 3_level_8  Unnamed: 4_level_8  Unnamed: 5_level_8   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_10 Unnamed: 4_level_10 Unnamed: 5_level_10   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_12 Unnamed: 4_level_12 Unnamed: 5_level_12   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_14 Unnamed: 4_level_14 Unnamed: 5_level_14   
                    W-L               AdjEM                AdjO   
    Unnamed: 3_level_16 Unnamed: 4_level_16 Unnamed: 5_level_16   
                    W-L               AdjEM                AdjO   
0                  26-0              +36.59               126.0   
1                  24-6              +30.70               118.8   
2                  20-4              +30.50               119.3   
3                  23-2              +29.93               123.0   
4                  25-3              +29.68               119.6   
..                  ...                 ...                 ...   
368                1-17              -25.02                87.1   
369                4-14              -26.12                84.2   
370                 0-0              -27.33                83.9   
371                 0-9              -29.52                88.3   
372                2-22              -41.51                79.2   

     Unnamed: 6_l

In [26]:
df['Unnamed: 4_level_0']

,AdjEM
,Unnamed: 4_level_2
,AdjEM
,Unnamed: 4_level_4
,AdjEM
,Unnamed: 4_level_6
,AdjEM
,Unnamed: 4_level_8
,AdjEM
,Unnamed: 4_level_10
,AdjEM


In [21]:
df['Unnamed: 4_level_0',
 'AdjEM',
 'Unnamed: 4_level_2',
 'AdjEM',
 'Unnamed: 4_level_4',
 'AdjEM',
 'Unnamed: 4_level_6',
 'AdjEM',
 'Unnamed: 4_level_8',
 'AdjEM',
 'Unnamed: 4_level_10',
 'AdjEM',
 'Unnamed: 4_level_12',
 'AdjEM',
 'Unnamed: 4_level_14',
 'AdjEM',
 'Unnamed: 4_level_16',
 'AdjEM']

0      +36.59
1      +30.70
2      +30.50
3      +29.93
4      +29.68
        ...  
368    -25.02
369    -26.12
370    -27.33
371    -29.52
372    -41.51
Name: (Unnamed: 4_level_0, AdjEM, Unnamed: 4_level_2, AdjEM, Unnamed: 4_level_4, AdjEM, Unnamed: 4_level_6, AdjEM, Unnamed: 4_level_8, AdjEM, Unnamed: 4_level_10, AdjEM, Unnamed: 4_level_12, AdjEM, Unnamed: 4_level_14, AdjEM, Unnamed: 4_level_16, AdjEM), Length: 373, dtype: object